In [93]:
import os
import re
import time
import imageio
import json
import glob
from pathlib import Path
from datetime import datetime, timedelta

import geojsoncontour
import numpy as np
import pandas as pd

from netCDF4 import Dataset as NetCDFFile
import matplotlib.pyplot as plt

from wrf import getvar, interplevel, to_np, latlon_coords

import plotly.graph_objects as go

In [94]:
nc_file = NetCDFFile('../wrf_output')
time_size = nc_file.dimensions['Time'].size

In [43]:
# https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html
height = getvar(nc_file, 'height', timeidx=1)
Var_all = getvar(nc_file, 'uvmet', timeidx=1)

var = interplevel(Var_all, height, 200)

(lats, lons) = latlon_coords(var)

var

<xarray.DataArray 'uvmet_interp' (u_v: 2, south_north: 81, west_east: 90)>
array([[[ -7.55934824,  -7.56917488,  -7.51826835, ...,  -4.5772625 ,
          -4.51960919,  -4.46170187],
        [ -7.71850547,  -7.82084551,  -7.74178332, ...,  -4.50715206,
          -4.39686461,  -4.43734628],
        [ -7.90277856,  -8.04899261,  -7.93142464, ...,  -4.53872797,
          -4.45769363,  -4.53082091],
        ...,
        [-15.67491216, -15.70389952, -15.59432746, ..., -13.47292444,
         -13.38913539, -13.44210736],
        [-15.62661081, -15.60758709, -15.53670559, ..., -13.67253107,
         -13.58707035, -13.59854442],
        [-15.60734558, -15.60253764, -15.59763788, ..., -13.88872178,
         -13.84784565, -13.80640844]],

       [[ -8.16623686,  -8.15241949,  -8.22205447, ...,  -4.27580211,
          -4.11187583,  -3.88263657],
        [ -8.28793413,  -8.32634391,  -8.43018876, ...,  -4.37823583,
          -4.18948275,  -3.94053697],
        [ -8.3610525 ,  -8.36608992,  -8.29593607, ...,  -4.34086884,
          -4.15911553,  -3.99817961],
        ...,
        [-12.50080138, -11.86773202, -11.57554734, ...,  -9.1401179 ,
          -9.11987127,  -9.08266141],
        [-12.51149916, -12.08513442, -11.97405731, ...,  -9.13738789,
          -9.13056554,  -9.16491188],
        [-12.5214955 , -12.36473324, -12.34295396, ...,  -9.29381453,
          -9.2447472 ,  -9.24694503]]])
Coordinates:
  * u_v      (u_v) <U1 'u' 'v'
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-03-30T03:00:00
    level    int64 200
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          m s-1
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LatLon(stand_lon=-74.8219985961914, moad_cen_lat=10.98099...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     m

In [82]:
var[0, 0]

<xarray.DataArray 'uvmet_interp' (west_east: 90)>
array([-7.55934824, -7.56917488, -7.51826835, -7.48767638,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan, -7.20556047, -7.1755945 , -7.14749149,
       -7.11650507, -7.0866823 , -7.05804811, -7.04230501, -7.02605987,
       -6.98706671, -6.95362131, -6.93315473, -6.9016476 , -6.86868815,
       -6.85103765, -6.82982791, -6.80206166, -6.77982957, -6.75605869,
       -6.7294319 , -6.70482799, -6.68054606, -6.65195321, -6.6224457 ,
       -6.59620266, -6.57484103, -6.5526593 , -6.5231452 , -6.49399328,
       -6.46761101, -6.44262507, -6.4186512 , -6.39053833, -6.35780079,
       -6.32720156, -6.3023255 , -6.2754084 , -6.24535135, -6.21311961,
       -6.1778357 , -6.14515589, -6.11490055, -6.08375955, -6.05254836,
       -6.02131988, -5.98823531, -5.95310133, -5.91821561, -5.88490178,
       -5.85093185, -5.81437677, -5.77818527, -5.74206241, -5.70252113,
       -5.66328069, -5.62414661, -5.58111932, -5.53732022, -5.49515807,
       -5.45332555, -5.40946566, -5.36215759, -5.31401463, -5.26622968,
       -5.2179641 , -5.16855443, -5.11829902, -5.0670648 , -5.01507447,
       -4.96287478, -4.91001418, -4.85594584, -4.80089607, -4.74545209,
       -4.68984695, -4.63410162, -4.5772625 , -4.51960919, -4.46170187])
Coordinates:
    u_v      <U1 'u'
    XLONG    (west_east) float32 -74.96 -74.96 -74.96 ... -74.69 -74.68 -74.68
    XLAT     (west_east) float32 10.85 10.85 10.85 10.85 ... 10.85 10.85 10.85
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-03-30T03:00:00
    level    int64 200
Dimensions without coordinates: west_east
Attributes:
    FieldType:      104
    units:          m s-1
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LatLon(stand_lon=-74.8219985961914, moad_cen_lat=10.98099...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     m

In [115]:
def get_data(nc_file: NetCDFFile, timeidx:int):
    dx = 150
    height = getvar(nc_file, 'height', timeidx=timeidx)

    u_all = getvar(nc_file, 'ua', timeidx=timeidx)
    v_all = getvar(nc_file, 'va', timeidx=timeidx)
    w_all = getvar(nc_file, 'wa', timeidx=timeidx)
    T_all = getvar(nc_file, 'tc', timeidx=timeidx)
    P_all = getvar(nc_file, 'pressure', timeidx=timeidx)

    P = interplevel(P_all, height, dx)
    T = interplevel(T_all, height, dx)
    u = interplevel(u_all, height, dx)
    v = interplevel(v_all, height, dx)
    w = interplevel(w_all, height, dx)
    
    data = {
        'wind': ('Wind speed (m/s)', np.sqrt(u ** 2 + v ** 2)),
        'temp': ('Temperature (C)', T),
        'uwind': ('U wind speed (m/s)', u),
        'vwind': ('V wind speed (m/s)', v),
        'press': ('Pressure (hPa)', P)
    }

    return data

def geojson_title_to_float(title):
    result = re.search(r"([-]?([0-9]*[.])?[0-9]+)-([-]?([0-9]*[.])?[0-9]+)", title)
    groups = result.groups()
    
    value = np.median([float(groups[0]), float(groups[2])])
    
    return value

def gj_to_df(gj):
    gj_data = np.zeros([len(gj['features']), 2])

    for i in range(len(gj['features'])):
        gj['features'][i]['id'] = i
        gj_data[i, 0] = i
        gj_data[i, 1] = geojson_title_to_float(gj['features'][i]['properties']['title'])

    df = pd.DataFrame(gj_data, columns=['id', 'variable'])
    
    return df


def build_fig(lats, lons, caption, variable, date):
    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet)

    gj = json.loads(geojsoncontour.contourf_to_geojson(contourf=contour, ndigits=4, unit='m'))
    df_contour = gj_to_df(gj)
    
    zmin = df_contour.variable.min() - df_contour.variable.median() / 10
    zmax = df_contour.variable.max() + df_contour.variable.median() / 10

    trace = go.Choroplethmapbox(
        geojson=gj,
        locations=df_contour.id,
        z=df_contour.variable,
        zmin=zmin,
        zmax=zmax,
        colorscale='jet',
        marker_line_width=0.1,
        marker=dict(opacity=0.2)
    )
    
    layout = go.Layout(
        title=f"{caption} - {date} GMT-5",
        title_x=0.5,
        width=600,
        margin=dict(t=26, b=0, l=0, r=0),
        font=dict(color='black', size=10),
        mapbox = dict(
            center = dict(
                lat = lats.mean().item(0),
                lon = lons.mean().item(0)
            ),
            zoom=11,
            style='carto-positron'
        )
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    
    return fig

def get_image(timeidx: int, nc_var: str, start_date: datetime):
    date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)

    data = get_data(nc_file, timeidx)

    (caption, variable) = data[nc_var]
    (lats, lons) = latlon_coords(variable)
    
    fig = build_fig(lats, lons, caption, variable, date)
    
    png_file = f"{nc_var}_{timeidx}.png"
    try:
        fig.write_image(png_file)
    except Exception:
        return None
    
    img = imageio.imread(png_file)
    os.remove(png_file)

    return img

In [118]:
nc_var = 'press'
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')

results = [get_image(timeidx, nc_var, start_date) for timeidx in range(time_size)]

imageio.mimwrite(f"{nc_var}_test.gif", [img for img in results if img is not None], fps=0.5);

In [112]:
fig = get_image(2, 'press', start_date)

    id  variable
0  0.0    992.36
1  1.0    992.44
2  2.0    992.52
3  3.0    992.60
4  4.0    992.68
5  5.0    992.76
6  6.0    992.84
7  7.0    992.92


ValueError: Transform failed with error code 525: Cannot read property 'type' of undefined

In [56]:
asdf

NameError: name 'asdf' is not defined

In [11]:
ds = xr.open_dataset('../wrf_output')
ds.coords

Coordinates:
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
    XTIME    (Time) datetime64[ns] ...
    XLAT_U   (Time, south_north, west_east_stag) float32 ...
    XLONG_U  (Time, south_north, west_east_stag) float32 ...
    XLAT_V   (Time, south_north_stag, west_east) float32 ...
    XLONG_V  (Time, south_north_stag, west_east) float32 ...

In [21]:
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')
timeidx = 1

date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)

data = get_data(nc_file, timeidx)

(caption, U) = data['uwind']
(caption, V) = data['uwind']
(lats, lons) = latlon_coords(V)

In [45]:
contour = plt.contourf(lons, lats, U, cmap=plt.cm.jet)
gj = json.loads(geojsoncontour.contourf_to_geojson(contourf=contour, ndigits=4, unit='m'))
u_df_contour = gj_to_df(gj)

In [46]:
contour = plt.contourf(lons, lats, V, cmap=plt.cm.jet)
gj = json.loads(geojsoncontour.contourf_to_geojson(contourf=contour, ndigits=4, unit='m'))
v_df_contour = gj_to_df(gj)

In [ ]:
gj_data = np.zeros([len(gj['features']), 2])

for i in range(len(gj['features'])):
    gj['features'][i]['id'] = i
    gj_data[i, 0] = i
    gj_data[i, 1] = geojson_title_to_float(gj['features'][i]['properties']['title'])
    

In [50]:
f = ff.create_quiver(lats, lons, U, V,
                       scale=.6,
                       arrow_scale=.5,
                       name='quiver',
                       line_width=2, line_color='black')

KeyboardInterrupt: 

In [ ]:
layout = go.Layout(
    title=f"{caption} - {date}",
    title_x=0.5,
    width=600,
    margin=dict(t=26, b=0, l=0, r=0),
    font=dict(color='dark grey', size=10),
    mapbox = dict(
        center = dict(
            lat = lats.mean().item(0),
            lon = lons.mean().item(0)
        ),
        zoom=11,
        style='carto-positron'
    )
)

fig = go.Figure(data=[f], layout=layout)

fig.show()